# Packages Used
* Import Statements and Capcha Avoiding Code

In [2]:
import os
import requests
from bs4 import BeautifulSoup as BS
from urllib.request import urlopen
import urllib.parse
import urllib.error
import ssl
import sys
from urllib.request import Request, urlopen
import pandas as pd
import numpy as np
import re
import json
from pandas.io.json._normalize import nested_to_record
ctx = ssl.create_default_context()
ctx.check_hostname=False
ctx.verify_mode=ssl.CERT_NONE
import warnings
warnings.filterwarnings("ignore")

# Add City wide pages available in  BayArea
* Each City define a variable with properties to collect from  each cities pages from zillow

In [4]:
fremont_sigle_family_1="https://www.zillow.com/fremont-ca/houses/1_p/"
fremont_sigle_family_2="https://www.zillow.com/fremont-ca/houses/2_p/"
santaclara_single_family="https://www.zillow.com/homes/Santa-Clara,-CA_rb/"
sj1="https://www.zillow.com/homes/sanjose-ca_rb/1_p"
sj2="https://www.zillow.com/homes/sanjose-ca_rb/2_p"
sj3="https://www.zillow.com/homes/sanjose-ca_rb/3_p"
sj4="https://www.zillow.com/homes/sanjose-ca_rb/4_p"
sj5="https://www.zillow.com/homes/sanjose-ca_rb/5_p"
sj6="https://www.zillow.com/homes/sanjose-ca_rb/6_p"
sj7="https://www.zillow.com/homes/sanjose-ca_rb/7_p"
sj8="https://www.zillow.com/homes/sanjose-ca_rb/8_p"
sfo_1="https://www.zillow.com/homes/San-Francisco,-CA_rb/1_p"
sfo_2="https://www.zillow.com/homes/San-Francisco,-CA_rb/2_p"
sfo_3="https://www.zillow.com/homes/San-Francisco,-CA_rb/3_p"
sfo_4="https://www.zillow.com/homes/San-Francisco,-CA_rb/4_p"
sfo_5="https://www.zillow.com/homes/San-Francisco,-CA_rb/5_p"
sfo_6="https://www.zillow.com/homes/San-Francisco,-CA_rb/6_p"
sfo_7="https://www.zillow.com/homes/San-Francisco,-CA_rb/7_p"
sfo_8="https://www.zillow.com/homes/San-Francisco,-CA_rb/8_p"
sfo_9="https://www.zillow.com/homes/San-Francisco,-CA_rb/9_p"
dublin="https://www.zillow.com/homes/Dublin,-CA_rb/"
pleaston="https://www.zillow.com/homes/Pleasanton,-CA_rb/"
livermore_1="https://www.zillow.com/livermore-ca/houses/1_p/"
livermore_2="https://www.zillow.com/livermore-ca/houses/2_p/"
sunnyvale_1="https://www.zillow.com/homes/Sunnyvale,-CA_rb/1_p/"
sunnyvale_2="https://www.zillow.com/homes/Sunnyvale,-CA_rb/2_p/"
mountainview="https://www.zillow.com/mountain-view-ca/houses/1_p"
sanroman="https://www.zillow.com/homes/San-Ramon,-CA_rb/1_p"
oakland_1="https://www.zillow.com/homes/Oakland,-CA_rb/1_p/"
oakland_2="https://www.zillow.com/homes/Oakland,-CA_rb/2_p/"
oakland_3="https://www.zillow.com/homes/Oakland,-CA_rb/3_p/"
oakland_4="https://www.zillow.com/homes/Oakland,-CA_rb/4_p/"
oakland_5="https://www.zillow.com/homes/Oakland,-CA_rb/5_p/"
oakland_6="https://www.zillow.com/homes/Oakland,-CA_rb/6_p/"
oakland_7="https://www.zillow.com/homes/Oakland,-CA_rb/7_p/"
hayward_1="https://www.zillow.com/homes/Hayward,-CA_rb/"
hayward_2="https://www.zillow.com/hayward-ca/2_p"
hayward_3="https://www.zillow.com/hayward-ca/3_p"



urls = [fremont_sigle_family_1,fremont_sigle_family_2, santaclara_single_family,
sfo_1,sfo_2,sfo_3,sfo_4,sfo_5,sfo_6,sfo_7,sfo_8,sfo_9, sj1,sj2,sj3,sj4,sj5,sj6,sj7,sj8,dublin,pleaston,livermore_1,livermore_2,sunnyvale_1,sunnyvale_2,mountainview,
oakland_1,oakland_2,oakland_3,oakland_4,oakland_5,oakland_6,oakland_7,hayward_1,hayward_2,hayward_3]

properties = []

# Collect all property links
* For each city page scrape the list of properties available and add the unique link per property into a list

In [5]:
#Gather Property Links
for url in urls:
    links = []
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9 '})
    webpage = urlopen(req).read()
    soup = BS(webpage, features="html.parser")
    pages = re.findall(r"https:\/\/www\.zillow\.com\/homedetails\/\S+\/\d+_zpid\/",str(soup))
    #print(pages)
    links = np.unique([link for link in pages if (link.startswith('https://www.zillow.com/homedetails/') & ('image' not in link))])
    # print(links)
    for lk in links:
        properties.append(lk)

## Dumping Property List
+ Used Unique function to remove any duplicate properties
+ Push the data to a data frame
+ Dump the data into Excel for future Reference

In [10]:
properties = np.unique(properties)
print(len(properties))

1326


In [14]:
listProperties = pd.DataFrame(properties)
listProperties.to_excel("zillow_properties_dump.xlsx")

# Scrapping Data
* Take each property in the list
* Make a request using request object
* Using Beautiful Soup Package capture the JSON present in hdpApolloPreloadedData element
* Parse the Json using json package and extract what is exactly required
* Since the Json is nested used a nested_to_record library to denormalize the chaining in the JSON (Flatten it)
* Add it to the Dataframe

In [ ]:
data_dump = pd.DataFrame()
for prop in properties:
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9 '})
    webpage = urlopen(req).read()
    soup = BS(webpage, features="html.parser")
    s = soup.find('script', id='hdpApolloPreloadedData', type='application/json').string
    full_json = json.loads(s)
    apiCache = full_json['apiCache']
    VariantQuery = json.loads(apiCache)
    varient_list = list(VariantQuery.values())[1]
    pds = varient_list['property']
    flat = nested_to_record(pds, sep='_')
    data_dump = data_dump.append(flat, ignore_index=True)


# Dump the data to Excel 
* Print the length of dataframe to check if all properties where scrapped
* Dump the the data to excel
* we will see a error saying characters since it exceeds Excel's limit for URLS. This is not a problem, data is written to excel

In [ ]:
print(len(data_dump))
data_dump.head(100)
data_dump[['price','zestimate','zpid','mlsid','rentZestimate']]
data_dump.to_excel("zillow_dump_full_data.xlsx")

# Data Minification
* Since the data collected is huge, the no of columns is around 750
* We short listed around 200 Columns with cleaner data into a different Dataframe data_minfied
* Dump the data into a excel for all the analysis and regression

In [37]:
pd.set_option('display.max_columns', None)
data_minfied = data_dump[[ 'stateId', 'countyId', 'cityId', 'zpid', 'mlsid', 'streetAddress', 'city', 'county', 'state', 'zipcode', 'price', 'zestimate', 'rentZestimate', 'yearBuilt', 'homeType', 'homeStatus', 'bedrooms', 'bathrooms', 'longitude', 'latitude', 'hasBadGeocode', 'description', 'livingAreaValue', 'livingAreaUnits', 'lotAreaValue', 'lotAreaUnits', 'priceHistory', 'lastSoldPrice', 'zestimateLowPercent', 'zestimateHighPercent', 'restimateLowPercent', 'restimateHighPercent', 'hideZMA',
 'dateSoldString', 'livingAreaUnitsShort', 'taxAssessedValue', 'taxAssessedYear', 'isPreforeclosureAuction', 'monthlyHoaFee', 'propertyTaxRate', 'timeZone', 'whatILove', 'isFeatured', 'isListedByOwner', 'isCommunityPillar', 'taxHistory', 'isIncomeRestricted', 'homeValues', 'buildingId', 'daysOnZillow', 'schools', 'zillowOfferMarket_code', 'address_streetAddress', 'address_city', 'address_state', 'address_zipcode', 'address_neighborhood', 'address_community', 'address_subdivision', 'zestimateDeepDiveData_zmaModel_approveDate', 'resoFacts_atAGlanceFacts', 'resoFacts_bedrooms', 'resoFacts_bathrooms', 'resoFacts_bathroomsFull', 'resoFacts_bathroomsThreeQuarter', 'resoFacts_bathroomsHalf', 'resoFacts_bathroomsOneQuarter', 'resoFacts_bathroomsPartial', 'resoFacts_mainLevelBathrooms', 'resoFacts_rooms', 'resoFacts_basement', 'resoFacts_flooring', 'resoFacts_heating', 'resoFacts_hasHeating', 'resoFacts_cooling', 'resoFacts_hasCooling', 'resoFacts_appliances', 'resoFacts_laundryFeatures', 'resoFacts_fireplaces', 'resoFacts_fireplaceFeatures', 'resoFacts_hasFireplace', 'resoFacts_furnished', 'resoFacts_commonWalls', 'resoFacts_buildingArea', 'resoFacts_livingArea', 'resoFacts_aboveGradeFinishedArea', 'resoFacts_belowGradeFinishedArea', 'resoFacts_virtualTour', 'resoFacts_parking', 'resoFacts_parkingFeatures', 'resoFacts_garageSpaces', 'resoFacts_coveredSpaces', 'resoFacts_hasAttachedGarage', 'resoFacts_hasGarage', 'resoFacts_openParkingSpaces', 'resoFacts_hasOpenParking', 'resoFacts_carportSpaces', 'resoFacts_hasCarport', 'resoFacts_otherParking', 'resoFacts_accessibilityFeatures', 'resoFacts_levels', 'resoFacts_stories', 'resoFacts_entryLevel', 'resoFacts_entryLocation', 'resoFacts_hasPrivatePool', 'resoFacts_hasSpa', 'resoFacts_spaFeatures', 'resoFacts_exteriorFeatures', 'resoFacts_patioAndPorchFeatures', 'resoFacts_fencing', 'resoFacts_view', 'resoFacts_hasView', 'resoFacts_hasWaterfrontView', 'resoFacts_waterfrontFeatures', 'resoFacts_waterBodyName', 'resoFacts_frontageType', 'resoFacts_frontageLength', 'resoFacts_topography', 'resoFacts_woodedArea', 'resoFacts_vegetation', 'resoFacts_canRaiseHorses', 'resoFacts_lotSize', 'resoFacts_lotSizeDimensions', 'resoFacts_otherStructures', 'resoFacts_additionalParcelsDescription', 'resoFacts_hasAdditionalParcels', 'resoFacts_parcelNumber', 'resoFacts_hasAttachedProperty', 'resoFacts_hasLandLease', 'resoFacts_landLeaseAmount', 'resoFacts_zoning', 'resoFacts_zoningDescription', 'resoFacts_homeType', 'resoFacts_architecturalStyle', 'resoFacts_constructionMaterials', 'resoFacts_foundationDetails', 'resoFacts_roofType', 'resoFacts_windowFeatures', 'resoFacts_propertyCondition', 'resoFacts_isNewConstruction', 'resoFacts_yearBuilt', 'resoFacts_developmentStatus', 'resoFacts_yearBuiltEffective', 'resoFacts_onMarketDate', 'resoFacts_builderModel', 'resoFacts_builderName', 'resoFacts_hasHomeWarranty', 'resoFacts_electric', 'resoFacts_hasElectricOnProperty', 'resoFacts_gas', 'resoFacts_sewer', 'resoFacts_waterSources', 'resoFacts_utilities', 'resoFacts_greenBuildingVerificationType', 'resoFacts_greenEnergyEfficient', 'resoFacts_greenIndoorAirQuality', 'resoFacts_greenSustainability', 'resoFacts_greenWaterConservation', 'resoFacts_numberOfUnitsInCommunity', 'resoFacts_numberOfUnitsVacant', 'resoFacts_storiesTotal', 'resoFacts_allowedPets', 'resoFacts_hasPetsAllowed', 'resoFacts_hasRentControl', 'resoFacts_buildingFeatures', 'resoFacts_structureType', 'resoFacts_buildingName', 'resoFacts_elementarySchool', 'resoFacts_elementarySchoolDistrict', 'resoFacts_middleOrJuniorSchool', 'resoFacts_middleOrJuniorSchoolDistrict', 'resoFacts_highSchool', 'resoFacts_highSchoolDistrict', 'resoFacts_securityFeatures', 'resoFacts_communityFeatures', 'resoFacts_isSeniorCommunity', 'resoFacts_cityRegion', 'resoFacts_associationFee', 'resoFacts_hasAssociation', 'resoFacts_associationAmenities', 'resoFacts_associationFeeIncludes', 'resoFacts_associationName', 'resoFacts_associationPhone', 'resoFacts_associationFee2', 'resoFacts_associationName2', 'resoFacts_associationPhone2', 'resoFacts_taxAnnualAmount', 'resoFacts_listingId', 'resoFacts_buildingAreaSource', 'resoFacts_specialListingConditions', 'resoFacts_availabilityDate', 'resoFacts_basementYN', 'resoFacts_buyerAgencyCompensation', 'resoFacts_buyerAgencyCompensationType', 'resoFacts_doorFeatures', 'resoFacts_interiorFeatures', 'resoFacts_leaseTerm', 'resoFacts_lotFeatures', 'resoFacts_mainLevelBedrooms', 'resoFacts_otherEquipment', 'resoFacts_poolFeatures', 'resoFacts_propertySubType', 'resoFacts_virtualTourURLUnbranded', 'resoFacts_waterViewYN', 'resoFacts_waterView', 'resoFacts_livingAreaRange', 'resoFacts_livingAreaRangeUnits', 'resoFacts_exclusions', 'resoFacts_landLeaseExpirationDate', 'resoFacts_subdivisionName', 'resoFacts_attic', 'resoFacts_bodyType', 'resoFacts_cropsIncludedYN', 'resoFacts_elevation', 'resoFacts_elevationUnits', 'resoFacts_foundationArea', 'resoFacts_horseAmenities', 'resoFacts_horseYN', 'resoFacts_inclusions', 'resoFacts_incomeIncludes', 'resoFacts_irrigationWaterRightsAcres', 'resoFacts_irrigationWaterRightsYN', 'resoFacts_listingTerms', 'resoFacts_ownership', 'resoFacts_ownershipType', 'resoFacts_roadSurfaceType', 'resoFacts_tenantPays', 'resoFacts_totalActualRent', 'resoFacts_waterSource', 'resoFacts_petsMaxWeight', 'resoFacts_municipality', 'resoFacts_pricePerSquareFoot', 'resoFacts_marketingType', 'resoFacts_depositsAndFees', 'resoFacts_daysOnZillow', 'resoFacts_hoaFee', 'resoFacts_offerReviewDate', 'resoFacts_otherFacts', 'resoFacts_listAOR', 'solarPotential_sunScore', 'solarPotential_buildFactor', 'solarPotential_climateFactor', 'solarPotential_electricityFactor', 'solarPotential_solarFactor', 'mortgageRates_thirtyYearFixedRate', 'mortgageRates_fifteenYearFixedRate', 'mortgageRates_arm5Rate', 'parentRegion_name']]

In [38]:
len(data_minfied)

1325

In [39]:
data_minfied.to_excel("zillow_dump_data_minfied.xlsx")